# Assignment



1. use pandas read_csv with sep='\t' to read in the following 2 files available from the us naval academy:
- url = 'https://www.usna.edu/Users/cs/nchamber/data/twitter/keyword-tweets.txt'
- url = 'https://www.usna.edu/Users/cs/nchamber/data/twitter/general-tweets.txt'
<br/> <span style="color:red" float:right>[1 point]</span>


In [1]:
import string
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cProfile
import seaborn as sns

from scipy.sparse import coo_matrix # this is the sparse matrix format discussed in lecture

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix

In [2]:
import nltk
## A one-time requirement for these four downloads:
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('corpus')

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Error loading corpus: Package 'corpus' not found in index


In [3]:
import pandas as pd
import numpy as np

url = 'https://www.usna.edu/Users/cs/nchamber/data/twitter/keyword-tweets.txt'
tweet1 = pd.read_csv(url, sep = '\t', header = None)

url2 = 'https://www.usna.edu/Users/cs/nchamber/data/twitter/general-tweets.txt'
tweet2 = pd.read_csv(url2, sep = '\t', header = None)






2. concatenate these 2 data sets into a single data frame called LabeledTweets that has 2 columns, named Sentiment and Tweet <span style="color:red" float:right>[1 point]</span>


In [4]:
LabeledTweets = tweet1.append(tweet2, ignore_index=True)
print(len(tweet1)) # checked the length just to make sure it worked 
print(len(tweet2))
print(len(LabeledTweets))

LabeledTweets.columns =['Sentiment', 'Tweet']
LabeledTweets.head()

2004
2000
4004


,Sentiment,Tweet
0,POLIT,Global Voices Online Â» Alex Castro: A liberal...
1,POLIT,Do the Conservatives Have a Death Wish? http:/...
2,NOT,@MMFlint I've seen all of your movies and Capi...
3,POLIT,RT @AllianceAlert: * House Dems ask for civili...
4,POLIT,RT @AdamSmithInst Quote of the week: My politi...


3. replace sentiment labels 'POLIT': 1, 'NOT': 0; <span style="color:red" float:right>[0 point]</span>


I used .mask to replace the old values of POLIT AND NOT to 1 and zero. I then used .astype to change Sentiment into an interger. 

In [5]:
LabeledTweets.Sentiment = LabeledTweets.Sentiment.mask(LabeledTweets.Sentiment == "POLIT" , "1") 
LabeledTweets.Sentiment = LabeledTweets.Sentiment.mask(LabeledTweets.Sentiment == "NOT" , "0") 
LabeledTweets['Sentiment'] = LabeledTweets['Sentiment'].astype(int)
print(LabeledTweets.head()) #Used this as a check
print(LabeledTweets.dtypes) #Used this as a check 

   Sentiment                                              Tweet
0          1  Global Voices Online Â» Alex Castro: A liberal...
1          1  Do the Conservatives Have a Death Wish? http:/...
2          0  @MMFlint I've seen all of your movies and Capi...
3          1  RT @AllianceAlert: * House Dems ask for civili...
4          1  RT @AdamSmithInst Quote of the week: My politi...
Sentiment     int64
Tweet        object
dtype: object



4. clean the tweets
   1. remove all tokens that contain a "@". Remove the whole token, not just the character.
   2. remove all tokens that contain "http". Remove the whole token, not just the characters.
   3. **replace** (not remove) all punctuation marks with a space (" ")
   4. **replace** all numbers with a space
   5. **replace** all non ascii characters with a space
   7. convert all characters to lowercase
   8. strip extra whitespaces
   9. lemmatize tokens
   9. No need to remove stopwords because TfidfVectorizer will take care of that
<br/><span style="color:red" float:right>[9 point]</span>


Using the code given in class I changed it slightly to replace instead of remove where requested by the instructions. 

In [6]:
def preprocess(text, list_of_steps):
    for step in list_of_steps:
        if step == 'replace_non_ascii':
            text = ''.join([i if ord(i) < 128 else ' ' for i in text]) # we want to replace anything <128 with a space. ' ' indicates space. 
        elif step == 'lowercase':
            text = text.lower()
        elif step == 'replace_punctuation':
            text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
        elif step == 'replace_numbers':
            text = re.sub("\d+", " ", text)
        elif step == 'strip_whitespace':
            text = ' '.join(text.split())
        elif step == 'remove_at':
            word_list = text.split(' ')
            word_list = [item for item in word_list if '@' not in item and 'http' not in item] # we only want to keep words in the list that dont have this condition and then we reconnect the list
            text = ' '.join(word_list)
        elif step == 'stem_words':
            lmtzr = WordNetLemmatizer()
            word_list = text.split(' ')
            stemmed_words = [lmtzr.lemmatize(word) for word in word_list]
            text = ' '.join(stemmed_words)
    return text


steps = ['remove_at', 'lowercase', 'replace_punctuation', 'replace_numbers', 'strip_whitespace', 'replace_non_ascii', 'stem_words'] #'replace_punctuation', 'replace_numbers', 'strip_whitespace',]
#tweet_df['clean_tweet'] = tweet_df['tweet_text'].map(lambda s: preprocess(s, steps))
#cProfile.run("LabeledTweets['clean_tweet'] = LabeledTweets['Tweet'].map(lambda s: preprocess(s, steps))");
LabeledTweets['clean_tweet'] = LabeledTweets['Tweet'].map(lambda s: preprocess(s, steps))
display(LabeledTweets.head())
#print(string.punctuation)


,Sentiment,Tweet,clean_tweet
0,1,Global Voices Online Â» Alex Castro: A liberal...,global voice online alex castro a liberal l...
1,1,Do the Conservatives Have a Death Wish? http:/...,do the conservative have a death wish
2,0,@MMFlint I've seen all of your movies and Capi...,i ve seen all of your movie and capitalism is ...
3,1,RT @AllianceAlert: * House Dems ask for civili...,rt house dems ask for civility at town hall an...
4,1,RT @AdamSmithInst Quote of the week: My politi...,rt quote of the week my political opinion lean...



5. Use TfidfVectorizer from sklearn to prepare the data for machine learning.  Use max_features = 50;  <span style="color:red" float:right>[2 point]</span>


In [7]:
vectorizer = TfidfVectorizer(sublinear_tf = True, max_df = 0.5, max_features = 50, stop_words = 'english')
clean_texts = LabeledTweets['clean_tweet']
tf_idf_tweets = vectorizer.fit_transform(clean_texts)

tf_idf_tweets

<4004x50 sparse matrix of type '<class 'numpy.float64'>'
	with 6071 stored elements in Compressed Sparse Row format>

In [8]:
# type(LabeledTweets)
sentiment_list = LabeledTweets.Sentiment.values.tolist()


6. Use sklearn LogisticRegression to train a model on the  results on 75% of the data. <span style="color:red" float:right>[1 point]</span>


In [9]:
y_targets = np.array([y for y in sentiment_list])
X_train, X_test, y_train, y_test = train_test_split(tf_idf_tweets, y_targets, test_size = 3003, random_state = 42)

In [10]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression()


7. determine the accuracy on the training data and the test data.   Determine the baseline accuracy. <span style="color:red" float:right>[1 point]</span>


In [11]:
train_results = lr.predict(X_train)
test_results = lr.predict(X_test)

train_acc = np.mean(y_train == train_results)
test_acc = np.mean(y_test == test_results)

print('Train accuracy: {}'.format(train_acc))
print('Test accuracy: {}'.format(test_acc))
print('Baseline accuracy: {}'.format(np.max([np.mean(y_test == 1), np.mean(y_test == 0)])))

Train accuracy: 0.8961038961038961
Test accuracy: 0.8558108558108558
Baseline accuracy: 0.5704295704295704




8. Repeat steps 5, 6, and 7  with TfidfVectorizer max_features set to 5, 500, 5000, 50000 and discuss your accuracies. <span style="color:red" float:right>[2 point]</span>



When the max_feature is too small it wont have enough to work with and the accuracy wont be that good. Generally as you increase the number the fit will get better, but at a certain point there will be diminishing returns where a bigger number doesn't neccessiarly make a better fitting model. 

Max_features = 5 

In [12]:
vectorizer = TfidfVectorizer(sublinear_tf = True, max_df = 0.5, max_features = 5, stop_words = 'english')
clean_texts = LabeledTweets['clean_tweet']
tf_idf_tweets = vectorizer.fit_transform(clean_texts)

tf_idf_tweets

# type(LabeledTweets)
sentiment_list = LabeledTweets.Sentiment.values.tolist()


y_targets = np.array([y for y in sentiment_list])
X_train, X_test, y_train, y_test = train_test_split(tf_idf_tweets, y_targets, test_size = 3003, random_state = 42)

lr = LogisticRegression()
lr.fit(X_train, y_train)

train_results = lr.predict(X_train)
test_results = lr.predict(X_test)

train_acc = np.mean(y_train == train_results)
test_acc = np.mean(y_test == test_results)

print('Train accuracy: {}'.format(train_acc))
print('Test accuracy: {}'.format(test_acc))
print('Baseline accuracy: {}'.format(np.max([np.mean(y_test == 1), np.mean(y_test == 0)])))

Train accuracy: 0.7472527472527473
Test accuracy: 0.7379287379287379
Baseline accuracy: 0.5704295704295704


In [13]:
vectorizer = TfidfVectorizer(sublinear_tf = True, max_df = 0.5, max_features = 500, stop_words = 'english')
clean_texts = LabeledTweets['clean_tweet']
tf_idf_tweets = vectorizer.fit_transform(clean_texts)

tf_idf_tweets

# type(LabeledTweets)
sentiment_list = LabeledTweets.Sentiment.values.tolist()


y_targets = np.array([y for y in sentiment_list])
X_train, X_test, y_train, y_test = train_test_split(tf_idf_tweets, y_targets, test_size = 3003, random_state = 42)

lr = LogisticRegression()
lr.fit(X_train, y_train)

train_results = lr.predict(X_train)
test_results = lr.predict(X_test)

train_acc = np.mean(y_train == train_results)
test_acc = np.mean(y_test == test_results)

print('Train accuracy: {}'.format(train_acc))
print('Test accuracy: {}'.format(test_acc))
print('Baseline accuracy: {}'.format(np.max([np.mean(y_test == 1), np.mean(y_test == 0)])))

Train accuracy: 0.9400599400599401
Test accuracy: 0.8434898434898435
Baseline accuracy: 0.5704295704295704


In [14]:
vectorizer = TfidfVectorizer(sublinear_tf = True, max_df = 0.5, max_features = 5000, stop_words = 'english')
clean_texts = LabeledTweets['clean_tweet']
tf_idf_tweets = vectorizer.fit_transform(clean_texts)

tf_idf_tweets

# type(LabeledTweets)
sentiment_list = LabeledTweets.Sentiment.values.tolist()


y_targets = np.array([y for y in sentiment_list])
X_train, X_test, y_train, y_test = train_test_split(tf_idf_tweets, y_targets, test_size = 3003, random_state = 42)

lr = LogisticRegression()
lr.fit(X_train, y_train)

train_results = lr.predict(X_train)
test_results = lr.predict(X_test)

train_acc = np.mean(y_train == train_results)
test_acc = np.mean(y_test == test_results)

print('Train accuracy: {}'.format(train_acc))
print('Test accuracy: {}'.format(test_acc))
print('Baseline accuracy: {}'.format(np.max([np.mean(y_test == 1), np.mean(y_test == 0)])))

Train accuracy: 0.9690309690309691
Test accuracy: 0.8081918081918081
Baseline accuracy: 0.5704295704295704


In [15]:
vectorizer = TfidfVectorizer(sublinear_tf = True, max_df = 0.5, max_features = 50000, stop_words = 'english')
clean_texts = LabeledTweets['clean_tweet']
tf_idf_tweets = vectorizer.fit_transform(clean_texts)

tf_idf_tweets

# type(LabeledTweets)
sentiment_list = LabeledTweets.Sentiment.values.tolist()


y_targets = np.array([y for y in sentiment_list])
X_train, X_test, y_train, y_test = train_test_split(tf_idf_tweets, y_targets, test_size = 3003, random_state = 42)

lr = LogisticRegression()
lr.fit(X_train, y_train)

train_results = lr.predict(X_train)
test_results = lr.predict(X_test)

train_acc = np.mean(y_train == train_results)
test_acc = np.mean(y_test == test_results)

print('Train accuracy: {}'.format(train_acc))
print('Test accuracy: {}'.format(test_acc))
print('Baseline accuracy: {}'.format(np.max([np.mean(y_test == 1), np.mean(y_test == 0)])))

Train accuracy: 0.975024975024975
Test accuracy: 0.7928737928737929
Baseline accuracy: 0.5704295704295704


# End of assignment